In [9]:
from rdflib import Graph, Namespace, RDF, RDFS, XSD, BNode, Literal, OWL
import os

# Create a Graph
EX = Namespace("http://example.org/research_paper/")

# Create a graph
g = Graph()

# Bind common prefixes (for readability)
g.bind("ex", EX)
g.bind("rdfs", RDFS)
g.bind("rdf", RDF)
g.bind("xsd", XSD)
# g.bind("owl", OWL)

In [10]:
def create_property(g, namespace, property_name, domain, range_):
    g.add((namespace[property_name], RDF.type, RDF.Property))
    g.add((namespace[property_name], RDFS.domain, namespace[domain]))
    if range_ != "string" and range_ != "date":
        g.add((namespace[property_name], RDFS.range, namespace[range_]))
    else:
        if range_ == "date":
            g.add((namespace[property_name], RDFS.range, XSD.date))
        elif range_ == "string":
            g.add((namespace[property_name], RDFS.range, XSD.string))


def create_class(g, namespace, class_name):
    g.add((namespace[class_name], RDF.type, RDFS.Class))


def create_subclass(g, namespace, subclass, superclass):
    g.add((namespace[subclass], RDFS.subClassOf, namespace[superclass]))


In [11]:
print(f"Initial TBOX size: {len(g)} triples.")

Initial TBOX size: 0 triples.


In [12]:
# Create classes
create_class(g, EX, "Paper")
create_class(g, EX, "Author")
create_class(g, EX, "Owner")
create_class(g, EX, "Reviewer")
create_class(g, EX, "Journal")
create_class(g, EX, "Conference")
create_class(g, EX, "Topic")
create_class(g, EX, "Review")
create_class(g, EX, "Publication")
create_class(g, EX, "Edition")
create_class(g, EX, "Volume")

In [13]:
# Restrictions on classes
create_subclass(g, EX, "Owner", "Author")
create_subclass(g, EX, "Reviewer", "Author")

create_subclass(g, EX, "Conference", "Publication")
create_subclass(g, EX, "Journal", "Publication")


In [14]:
# Create properties
create_property(g, EX, "written_by", "Paper", "Author")
create_property(g, EX, "has_title", "Paper", 'string')
create_property(g, EX, "cites", "Paper", "Paper")
create_property(g, EX, "related_to", "Paper", "Topic")
create_property(g, EX, "corresponds_to", "Paper", "Owner")
create_property(g, EX, "has_review", "Paper", "Review")
create_property(g, EX, "note", "Reviewer", "Review")
create_property(g, EX, "published_in", "Paper", "Publication")
create_property(g, EX, "has_edition", "Conference", "Edition")
create_property(g, EX, "has_volume", "Journal", "Volume")
create_property(g, EX, "has_date_edition", "Edition", 'string')
create_property(g, EX, "has_date_volume", "Volume", 'string')
create_property(g, EX, "has_location", "Edition", 'string')
create_property(g, EX, "has_abstract", "Paper", 'string')

print(f"Finished TBOX generation. Total triples in TBOX: {len(g)}.")


Finished TBOX generation. Total triples in TBOX: 57.


In [15]:
# # Restriction: Paper must have exactly one corresponding author
# restriction1 = BNode()
# g.add((EX.Paper, OWL.equivalentClass, restriction1))
# g.add((restriction1, RDF.type, OWL.Restriction))
# g.add((restriction1, OWL.onProperty, EX.corresponds_to))
# g.add((restriction1, OWL.qualifiedCardinality, Literal(1, datatype=XSD.nonNegativeInteger)))
# g.add((restriction1, OWL.onClass, EX.Author))

# # Restriction: Paper must have exactly three reviews
# restriction2 = BNode()
# g.add((EX.Paper, OWL.equivalentClass, restriction2))
# g.add((restriction2, RDF.type, OWL.Restriction))
# g.add((restriction2, OWL.onProperty, EX.has_review))
# g.add((restriction2, OWL.qualifiedCardinality, Literal(3, datatype=XSD.nonNegativeInteger)))
# g.add((restriction2, OWL.onClass, EX.Review))

In [16]:
# Use current working directory instead of script path
script_dir = os.getcwd()  # Works in Jupyter
rdf_file_path = os.path.join(script_dir, "tbox.ttl")

# Save the graph
g.serialize(destination=rdf_file_path, format="turtle")

print(f"RDF saved to: {rdf_file_path}")

RDF saved to: c:\Users\usuario\Desktop\FIB\Ciència de Dades\2n Quadrimestre\SDM\Project\Knowledge Graph\SDM_lab2\tbox.ttl
